In [1]:
from htmd import *

Please cite -- HTMD: High-Throughput Molecular Dynamics for Molecular Discovery
J. Chem. Theory Comput., 2016, 12 (4), pp 1845-1852. 
http://pubs.acs.org/doi/abs/10.1021/acs.jctc.6b00049


New HTMD version (1.3.1) is available. You are currently on (1.0.26). Use 'conda update htmd' to update to the new version.


In [2]:
mol = Molecule('3PTB')
mol.filter("protein")

2016-06-30 19:13:22,981 - htmd.molecule.molecule - INFO - Attempting PDB query for 3PTB
2016-06-30 19:13:23,976 - htmd.molecule.molecule - INFO - Removed 72 atoms. 1629 atoms remaining in the molecule.


In [3]:
copied = mol.copy()
copied.filter('resname ARG LYS ASP GLU HIS CYS TYR')

2016-06-30 19:13:24,787 - htmd.molecule.molecule - INFO - Removed 1175 atoms. 454 atoms remaining in the molecule.


In [4]:
rid = np.unique(copied.get('resid')) # ia.resid also works!
rn = copied.get('resname', "name CA")

In [5]:
#He creado un monstruo... Os lo comento para que os sea mas facil de entender
#La idea es generar una pos_list (que contendra las interacciones entre aa basicos) y una neg_list (que contendra las interacciones entre aa acidos).
#He tomado el esquema que montó Mauro: 
pos_list = []
neg_list = []
for resname, resid in zip(rn, rid):
    if resname == "GLU":
        #primero, seleccionas los residuos que pueden interactuar con el residuo acido/basico de interes mediante puentes de hidrogeno
        a = mol.get("resid", sel="name O N OH OG1 OG OE1 NE2 ND2 OE2 SG ND1 NH1 OD1 OD2 NZ NH2 and within 4 of name OE1 OE2 and resid "+str(resid))
        #eliminamos resultados redundantes
        un = np.unique(a)
        for residueID in un:
            #Uso una arquitectura de try/except para curarme en salud
            try:
                l = len(np.unique(mol.get("resname", sel="resid "+str(residueID))))
                #Como he comentado en el face, hay posiciones de secuencia que contienen dos residuos, en lugar de uno.
                #Para esto, calculamos la distancia, si es mayor que 1 significa que tenemos posicion doble. En ese caso
                #itero sobre las posibles posiciones y si encuentro una que sea un acido (acido en este caso porque tratamos los acidos)
                #me la quedo y la añado al neg_list
                if l > 1:
                    for i in range(0, l):
                        if np.unique(mol.get("resname", sel="resid "+str(residueID)))[i] in ["GLU", "ASP"] and residueID != resid:
                            res = np.unique(mol.get("resname", sel="resid "+str(residueID)))
                            #Las dos listas generadas siguen un formato de tipo: resname y resid del residuo en el que estoy
                            #y resname y resid del residuo que interactua con el primero
                            tup = (resname, resid, res, residueID)
                            neg_list.append(tup)
                            print(np.unique(mol.get("resname", sel="resid "+str(residueID))), residueID)
                else:
                    #Si no hay posiciones con multiples aa, añado residuos a la lista con normalidad
                    if np.unique(mol.get("resname", sel="resid "+str(residueID))) in ["GLU", "ASP"] and residueID != resid:
                        res = np.unique(mol.get("resname", sel="resid "+str(residueID)))
                        tup = (resname, resid, res, residueID)
                        neg_list.append(tup)
                        print(np.unique(mol.get("resname", sel="resid "+str(residueID))), residueID)
            except:
                #En el caso de que hayan errores, por pantalla aparecera un print indicando en que posicion ocurren
                print("Error at " + np.unique(mol.get("resname", sel="resid "+str(residueID))), residueID)
                continue
                
    elif resname == "ASP":
        a = mol.get("resid", sel="name O N OH OG1 OG OE1 NE2 ND2 OE2 SG ND1 NH1 OD1 OD2 NZ NH2 and within 4 of name OD1 OD2 and resid "+str(resid))
        #print(np.unique(a), "resid", resid)
        un = np.unique(a)
        for residueID in un:
            try:
                l = len(np.unique(mol.get("resname", sel="resid "+str(residueID))))
                if l > 1:
                    for i in range(0, l):
                        if np.unique(mol.get("resname", sel="resid "+str(residueID)))[i] in ["GLU", "ASP"] and residueID != resid:
                            res = np.unique(mol.get("resname", sel="resid "+str(residueID)))
                            tup = (resname, resid, res, residueID)
                            neg_list.append(tup)
                            print(np.unique(mol.get("resname", sel="resid "+str(residueID))), residueID)
                else:
                    
                    if np.unique(mol.get("resname", sel="resid "+str(residueID))) in ["GLU", "ASP"] and residueID != resid:
                        res = np.unique(mol.get("resname", sel="resid "+str(residueID)))
                        tup = (resname, resid, res, residueID)
                        neg_list.append(tup)
                        print(np.unique(mol.get("resname", sel="resid "+str(residueID))), residueID)
            except:
                print("Error at " + np.unique(mol.get("resname", sel="resid "+str(residueID))), residueID)
                continue
                
    elif resname == "ARG":
        a = mol.get("resid", sel="name O N OH OG1 OG OE1 NE2 ND2 OE2 SG ND1 NH1 OD1 OD2 NZ NH2 and within 4 of name NH1 NH2 NE and resid "+str(resid))
        #print(np.unique(a), "resid", resid)
        un = np.unique(a)
        for residueID in un:
            try:
                l = len(np.unique(mol.get("resname", sel="resid "+str(residueID))))
                if l > 1:
                    for i in range(0, l):
                        if np.unique(mol.get("resname", sel="resid "+str(residueID)))[i] in ["LYS", "ARG"] and residueID != resid:
                            res = np.unique(mol.get("resname", sel="resid "+str(residueID)))
                            tup = (resname, resid, res, residueID)
                            pos_list.append(tup)
                            print(np.unique(mol.get("resname", sel="resid "+str(residueID))), residueID)
                else:
                    
                    if np.unique(mol.get("resname", sel="resid "+str(residueID))) in ["LYS", "ARG"] and residueID != resid:
                        res = np.unique(mol.get("resname", sel="resid "+str(residueID)))
                        tup = (resname, resid, res, residueID)
                        pos_list.append(tup)
                        print(np.unique(mol.get("resname", sel="resid "+str(residueID))), residueID)
            except:
                print("Error at " + np.unique(mol.get("resname", sel="resid "+str(residueID))), residueID)
                continue
                
    elif resname == "LYS":
        a = mol.get("resid", sel="name O N OH OG1 OG OE1 NE2 ND2 OE2 SG ND1 NH1 OD1 OD2 NZ NH2 and within 4 of name NZ and resid "+str(resid))
        #print(np.unique(a), "resid", resid)
        un = np.unique(a)
        for residueID in un:
            try:
                l = len(np.unique(mol.get("resname", sel="resid "+str(residueID))))
                if l > 1:
                    for i in range(0, l):
                        if np.unique(mol.get("resname", sel="resid "+str(residueID)))[i] in ["LYS", "ARG"] and residueID != resid:
                            res = np.unique(mol.get("resname", sel="resid "+str(residueID)))
                            tup = (resname, resid, res, residueID)
                            pos_list.append(tup)
                            print(np.unique(mol.get("resname", sel="resid "+str(residueID))), residueID)
                else:
                    
                    if np.unique(mol.get("resname", sel="resid "+str(residueID))) in ["LYS", "ARG"] and residueID != resid:
                        res = np.unique(mol.get("resname", sel="resid "+str(residueID)))
                        tup = (resname, resid, res, residueID)
                        pos_list.append(tup)
                        print(np.unique(mol.get("resname", sel="resid "+str(residueID))), residueID)
            except:
                print("Error at " + np.unique(mol.get("resname", sel="resid "+str(residueID))), residueID)
                continue

['ASP'] 71
['GLU'] 80
['GLU'] 70
['GLU'] 77


In [6]:
print(pos_list)

[]


In [7]:
print(neg_list)

[('GLU', 70, array(['ASP'], dtype=object), 71), ('GLU', 70, array(['GLU'], dtype=object), 80), ('GLU', 80, array(['GLU'], dtype=object), 70), ('GLU', 80, array(['GLU'], dtype=object), 77)]


In [8]:
def redundancy_eliminator(lista):
    #Ahora el objetivo es eliminar redundancia de esta lista
    #Para esto utilizaré listas: salidas contendra las posiciones para los residuos en los que estamos y llegadas contendras las
    #posiciones de los residuos con los que interactua. Eliminar contendrá los indices de los elementos de la lista que sean redundantes
    salidas = []
    llegadas = []
    eliminar = []
    if len(lista) > 1: #entonces existe la posibilidad de que tengamos redundancias -> aqui tenemos 80-70 y 70-80
        j = 0
        for i in range(0, len(lista)):
            if lista[i][1] in llegadas and lista[i][3] in salidas: #Entonces tendremos redundancia
                #Guardo los indices de aquellos elementos de la lista que sean redundantes
                eliminar.append(i)
                next
            else:
                #Aquellos elementos que no sean reduntantes los incorporo a listas para permitir su comparación en futuras iteraciones
                salidas.append(lista[i][1])
                llegadas.append(lista[i][3])

    for element in eliminar:
        lista.remove(lista[element])

    return lista
            

In [9]:
neg_list = redundancy_eliminator(neg_list)
print(neg_list)

[('GLU', 70, array(['ASP'], dtype=object), 71), ('GLU', 70, array(['GLU'], dtype=object), 80), ('GLU', 80, array(['GLU'], dtype=object), 77)]


In [10]:
pos_list = redundancy_eliminator(pos_list)
print(pos_list)

[]


In [11]:
def atom_density(resname, resid):

    #We will obtain scores for the two residues which are the same
    aa_score_acid = {"OH":1, "OG1":2, "OG":2, "NE2": 1, "ND1":1, "O": 2, "OD1": 2, "OE1": 2, "OE2": 2, "OD2": 2,"SG": -1, "N": -1, "NE": -1, "NE1": -1, "ND2": -2, "NH1": -2, "NH2": -2, "NZ": -3 }
    aa_score_base = {"OH":-1, "OG1":-1, "OG":-1, "NE2":-1, "ND1":-1, "O": 2, "OD1": 2, "OE1": 2, "OE2": 2, "OD2": 2,"SG": -1, "N": -1, "NE": -1, "NE1": -1, "ND2": -2, "NH1": -2, "NH2": -2, "NZ": -3 }
    score = 0
    a = []
    if resname == "ASP":
        a = mol.get("name", sel="name O N OH OG1 OG OE1 NE2 ND2 OE2 SG ND1 NH1 OD1 OD2 NZ NH2 and within 6 of name OD1 OD2 and resid "+str(resid))
        for element in a:
            try:
                score += aa_score_acid[element]
            except:
                continue

    elif resname == "GLU":
        a = mol.get("name", sel="name O N OH OG1 OG OE1 NE2 ND2 OE2 SG ND1 NH1 OD1 OD2 NZ NH2 and within 6 of name OE1 OE2 and resid "+str(resid))
        for element in a:
            try:
                score += aa_score_acid[element]
            except:
                continue

    elif resname == "LYS":
        a = mol.get("name", sel="name O N OH OG1 OG OE1 NE2 ND2 OE2 SG ND1 NH1 OD1 OD2 NZ NH2 and within 6 of name NZ and resid "+str(resid))
        for element in a:
            try:
                score += aa_score_base[element]
            except:
                continue

    elif resname == "ARG":
        a = mol.get("name", sel="name O N OH OG1 OG OE1 NE2 ND2 OE2 SG ND1 NH1 OD1 OD2 NZ NH2 and within 6 of name NH1 NH2 NE and resid "+str(resid))
        for element in a:
            try:
                score += aa_score_base[element]
            except:
                continue

    return score



In [12]:
#Ahora lo que toca es cambiar los nombre adecuadamente de la estructura original. 
#Primero los residuos negativos
for element in neg_list:
    if element[0] == "GLU" and element[2] == "ASP":
        print("jjjjjjjjj")
        mol.set("resname", "GLH", "resid "+str(element[1]))
    elif element[0] == "ASP" and element[2] == "GLU":
        print("ggggggggg")
        mol.set("resname", "GLH", "resid "+str(element[3]))
    elif element[0] == element[2]:
        print("uuuuuuuuuu")
        if atom_density(element[0], element[1]) > atom_density(element[2], element[3]): #el residuo con mayor electronegatividad a su al rededor se protona
            if element[0] == "GLU":
                name = "GLH"
            else:
                name = "ASH"
            mol.set("resname", name, "resid " + str(element[1]))
            print("aaaaaaaaaaaa", name, element[1], atom_density(element[0], element[1]))
            
        elif atom_density(element[0], element[1]) < atom_density(element[2], element[3]):
            if element[2] == "GLU":
                name = "GLH"
            else:
                name = "ASH"
            mol.set("resname", name, "resid " + str(element[3]))
            print("bbbbbbbbbbbbbbbbb", name, element[3], atom_density(element[2], element[3]))
            
        else:
            if element[2] == "GLU":
                name = "GLH"
            else:
                name = "ASH"
            mol.set("resname", name, "resid " + str(element[1]))
            print("cccccccccccccc", name, element[1])

jjjjjjjjj
uuuuuuuuuu
aaaaaaaaaaaa GLH 70 19
uuuuuuuuuu
aaaaaaaaaaaa GLH 80 18


In [15]:
for element in pos_list:
    if element[0] == "ARG" and element[2] == "LYS":
        mol.set("resname", "LYN", "resid "+str(element[3]))
    elif element[0] == "LYS" and element[2] == "ARG":
        mol.set("resname", "LYN", "resid "+str(element[1]))
    elif element[0] == element[2]:
        if atom_density(element[0], element[1]) < atom_density(element[2], element[3]): 
            #el residuo con menor electronegatividad a su al rededor se desprotona
            if element[0] == "LYS":
                name = "LYN"
            else:
                name = "AR0"
            mol.set("resname", name, "resid " + str(element[1]))
            
        elif atom_density(element[0], element[1]) > atom_density(element[2], element[3]):
            if element[2] == "LYS":
                name = "LYN"
            else:
                name = "AR0"
            mol.set("resname", name, "resid " + str(element[3]))
            
        else:
            if element[2] == "LYS":
                name = "LYN"
            else:
                name = "AR0"
            mol.set("resname", name, "resid " + str(element[1]))